# Preprocess content image, style image

In [1]:
from keras.preprocessing.image import load_img,img_to_array
import numpy as np
from keras.applications import vgg19
content_img_path="content5.jpg"
style_img_path="style3.jpg"

w,h=load_img(content_img_path).size
img_h=400
img_w=int(w*img_h/h)

def preprocess(path):
    img=load_img(path,target_size=(img_h,img_w))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=vgg19.preprocess_input(img)
    return img
def deprocess(img):
    img[:,:,0]+=103.939
    img[:,:,1]+=116.779
    img[:,:,2]+=123.68
    
    img=img[:,:,::-1]
    img=np.clip(img,0,255).astype("uint8")
    return img

Using TensorFlow backend.


# Define input, and load model

In [2]:
import keras.backend as K

content_img=K.constant(preprocess(content_img_path))
style_img=K.constant(preprocess(style_img_path))
combined_img=K.placeholder((1,img_h,img_w,3))

input_tensor=K.concatenate([content_img,
                           style_img,
                           combined_img],axis=0)
model=vgg19.VGG19(input_tensor=input_tensor,
                 weights="imagenet",
                 include_top=False)


# Define content loss

In [3]:
def content_loss(content,combined):
    return K.sum(K.square(combined-content))

# Define style loss

In [4]:
def gram_matrix(x):
    features=K.batch_flatten(K.permute_dimensions(x,(2,0,1)))
    gram=K.dot(features,K.transpose(features))
    return gram
def style_loss(style,combined):
    S=gram_matrix(style)
    C=gram_matrix(combined)
    channels=3
    size=img_h*img_w
    return K.sum(K.square(S-C))/(4.*(channels**2)*(size**2))

# Define total variation loss to get a more smoother image

In [5]:
def total_variation_loss(x):
    a=K.square(
        x[:,:img_h-1,:img_w-1,:]-x[:,1:,:img_w-1,:])
    b=K.square(
        x[:,:img_h-1,:img_w-1,:]-x[:,:img_h-1,1:,:])
    return K.sum(K.pow(a+b,1.25))


# Denfine the final loss to optimize

In [6]:
outputs_dict=dict([(layer.name,layer.output) for layer in model.layers])
content_layer="block5_conv2"
style_layers=["block1_conv1",
             "block2_conv1",
             "block3_conv1",
             "block4_conv1",
             "block5_conv1"]
total_variation_weight=1e-4
content_weight=0.025
style_weight=2

loss=K.variable(0.)
layer_features=outputs_dict[content_layer]
loss+=content_weight*content_loss(layer_features[0,:,:,:],layer_features[2,:,:,:])
for style_layer in style_layers:
    layer_feature=outputs_dict[style_layer]
    loss+=(style_weight/len(style_layers)*style_loss(layer_feature[1,:,:,:],layer_feature[2,:,:,:]))
loss+=total_variation_weight*total_variation_loss(combined_img)

# Define the optimize process

In [7]:
grads=K.gradients(loss,combined_img)[0]
fetch_loss_and_grads=K.function([combined_img],[loss,grads])

class Evaluator(object):
    def __init__(self):
        self.loss_value=None
        self.grads_value=None
    def loss(self,x):
        assert self.loss_value is None
        x=x.reshape((1,img_h,img_w,3))
        loss_value,grads_value=fetch_loss_and_grads([x])
        self.loss_value=loss_value
        self.grads_value=grads_value.flatten().astype("float64")
        return self.loss_value
    def grads(self,x):
        assert self.grads_value is not None
        grad_values=np.copy(self.grads_value)
        self.loss_value=None
        self.grads_value=None
        return grad_values
evaluator=Evaluator()

In [8]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time
result_prefix="my_result"
iterations=20
x=preprocess(content_img_path)
x=x.flatten()
for i in range(iterations):
    print("Its: %s"%i)
    start=time.time()
    x,min_val,info=fmin_l_bfgs_b(evaluator.loss,x,fprime=evaluator.grads,maxfun=20)
    print("Current loss :%s"%min_val)
    img=x.copy().reshape((img_h,img_w,3))
    img=deprocess(img)
    fname=result_prefix+"_at_iteration_%s.png"%i
    imsave(fname,img)
    print("Its:%s cost time %s"%(i,time.time()-start))

Its: 0
Current loss :9.71902e+09
Its:0 cost time 13.854782581329346
Its: 1


C:\Users\test07\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss :4.49846e+09
Its:1 cost time 12.120324850082397
Its: 2
Current loss :3.41534e+09
Its:2 cost time 11.618390798568726
Its: 3
Current loss :2.76415e+09
Its:3 cost time 11.63581919670105
Its: 4
Current loss :2.32921e+09
Its:4 cost time 11.651822328567505
Its: 5
Current loss :2.06511e+09
Its:5 cost time 11.71794605255127
Its: 6
Current loss :1.86816e+09
Its:6 cost time 11.619946002960205
Its: 7
Current loss :1.72587e+09
Its:7 cost time 11.568235158920288
Its: 8
Current loss :1.58093e+09
Its:8 cost time 11.639970064163208
Its: 9
Current loss :1.4662e+09
Its:9 cost time 11.679535150527954
Its: 10
Current loss :1.36686e+09
Its:10 cost time 11.667524337768555
Its: 11
Current loss :1.2845e+09
Its:11 cost time 11.566693305969238
Its: 12
Current loss :1.17939e+09
Its:12 cost time 11.531161069869995
Its: 13
Current loss :1.09848e+09
Its:13 cost time 11.663675546646118
Its: 14
Current loss :1.02845e+09
Its:14 cost time 11.747734785079956
Its: 15
Current loss :9.6793e+08
Its:15 cost time